In [0]:
from delta.tables import *

#Method_1: DeltaTable Builder API

In [0]:
from pyspark.sql.types import *


DeltaTable.create(spark)\
          .tableName("employee_demo")\
          .addColumn("emp_id",IntegerType(),False)\
          .addColumn("emp_name",StringType(),False)\
          .addColumn("gender",StringType(),False)\
          .addColumn("sal",IntegerType(),False)\
          .addColumn("dept",StringType(),False)\
          .property("description","table created for demo purpose")\
          .location("/FileStore/tables/delta/create_table")\
          .execute()

Out[3]: <delta.tables.DeltaTable at 0x7f64608d3dc0>

In [0]:
%sql


select * from employee_demo

emp_id,emp_name,gender,sal,dept


In [0]:
from pyspark.sql.types import *

#throws error as table already exists 

DeltaTable.create(spark)\
          .tableName("employee_demo")\
          .addColumn("emp_id",IntegerType(),False)\
          .addColumn("emp_name",StringType(),False)\
          .addColumn("gender",StringType(),False)\
          .addColumn("sal",IntegerType(),False)\
          .addColumn("dept",StringType(),False)\
          .property("description","table created for demo purpose")\
          .location("/FileStore/tables/delta/create_table")\
          .execute()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1388222939003278>:4
      1 from pyspark.sql.types import *
----> 4 DeltaTable.create(spark)\
      5           .tableName("employee_demo")\
      6           .addColumn("emp_id",IntegerType(),False)\
      7           .addColumn("emp_name",StringType(),False)\
      8           .addColumn("gender",StringType(),False)\
      9           .addColumn("sal",IntegerType(),False)\
     10           .addColumn("dept",StringType(),False)\
     11           .property("description","table created for demo purpose")\
     12           .location("/FileStore/tables/delta/create_table")\
     13           .execute()

File /databricks/spark/python/delta/tables.py:1438, in DeltaTableBuilder.execute(self)
   1429 @since(1.0)  # type: ignore[arg-type]
   1430 def execute(self) -> DeltaTable:
   1431     """
   1432     Execute Table Creatio

In [0]:
    
    #doesnt throw any error if table is already existing and the command just get skipped

    #if it doesnt exists already then new table is created
    
    DeltaTable.createIfNotExists(spark)\
          .tableName("employee_demo")\
          .addColumn("emp_id",IntegerType(),False)\
          .addColumn("emp_name",StringType(),False)\
          .addColumn("sal",IntegerType(),False)\
          .addColumn("dept",StringType(),False)\
          .property("description","table created for demo purpose")\
          .location("/FileStore/tables/delta/create_table")\
          .execute()

Out[7]: <delta.tables.DeltaTable at 0x7f646091e130>

In [0]:

#here we have modified the schema so even though the table exisits it got modified 

#even if table exists with the given name even though it treats as new one and old table gets truncated and deleted

DeltaTable.createOrReplace(spark)\
          .tableName("employee_demo")\
          .addColumn("emp_id",IntegerType(),False)\
          .addColumn("emp_name",StringType(),False)\
          .addColumn("sal",IntegerType(),False)\
          .addColumn("dept",StringType(),False)\
          .property("description","table created for demo purpose")\
          .location("/FileStore/tables/delta/create_table")\
          .execute()

Out[8]: <delta.tables.DeltaTable at 0x7f645ad2ddf0>

#Method_2 : SQL


#

In [0]:
%sql

/*THROWS ERROR AS TABLE ALREADY EXISTS*/
CREATE TABLE employee_demo
(
  emp_id INT,
  emp_name STRING,
  sal INT,
  dept STRING
)USING DELTA


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1388222939003282>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-1388222939003282>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("Q1JFQVRFIFRBQkxFIGVtcGxveWVlX2RlbW8KKAogIGVtcF9pZCBJTlQsCiAgZW1wX25hbWUgU1RSSU5HLAogIHNhbCBJTlQsCiAgZGVwdCBTVFJJTkcKKVVTSU5HIERFTFRB").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, functio

In [0]:
%sql
/*doesnt throw any error just skipps the command */
CREATE TABLE IF NOT EXISTS employee_demo
(
  emp_id INT,
  emp_name STRING,
  sal INT,
  dept STRING
)USING DELTA

In [0]:
%sql
/*creates a new table */
CREATE or replace TABLE  employee_demo
(
  emp_id INT,
  emp_name STRING,
  sal INT,
  dept STRING
)USING DELTA 
 LOCATION '/FileStore/tables/delta/create_table'

#METHOD 3: USING DATAFRAME

In [0]:
employee_data=[(1,"varun","M",13,"sales"),
               (2,"kusuma","F",10,"finance"),
               (3,"sahithi","F",12,"HR")
              ]

employee_schema=["id","emp_name","gender","hrly_pay","dept"] 

emp_df_1=spark.createDataFrame(employee_data,employee_schema)

In [0]:
emp_df_1.write.format('delta').option("path","/FileStore/tables/delta/sample_delta_tables/").saveAsTable('employee_demo_1')

In [0]:
%sql

select * from employee_demo_1

id,emp_name,gender,hrly_pay,dept
2,kusuma,F,10,finance
1,varun,M,13,sales
3,sahithi,F,12,HR


In [0]:
%fs

ls /FileStore/tables/delta/create_table/_delta_log/


path,name,size,modificationTime
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-0,.s3-optimization-0,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-1,.s3-optimization-1,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-2,.s3-optimization-2,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000000.crc,00000000000000000000.crc,2235,1717524476000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000000.json,00000000000000000000.json,1255,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000001.crc,00000000000000000001.crc,2144,1717525322000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000001.json,00000000000000000001.json,1149,1717525318000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000002.crc,00000000000000000002.crc,2094,1717527099000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000002.json,00000000000000000002.json,1049,1717527096000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000003.crc,00000000000000000003.crc,2108,1717527627000


In [0]:
%fs

head /FileStore/tables/delta/create_table/_delta_log/00000000000000000001.json

{"commitInfo":{"timestamp":1717525317222,"userId":"8432478468657958","userName":"paruchuri7781@gmail.com","operation":"CREATE OR REPLACE TABLE","operationParameters":{"isManaged":"false","description":null,"partitionBy":"[]","properties":"{\"description\":\"table created for demo purpose\"}"},"notebook":{"notebookId":"1388222939003274"},"clusterId":"0604-175509-m5964sr3","readVersion":0,"isolationLevel":"WriteSerializable","isBlindAppend":false,"operationMetrics":{},"engineInfo":"Databricks-Runtime/12.2.x-scala2.12","txnId":"15afdfaa-38ec-4c9f-8a6e-c6ac89fb8b39"}}
{"metaData":{"id":"e86fd596-9fbf-495e-b52e-fa5e7d513afa","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"emp_id\",\"type\":\"integer\",\"nullable\":false,\"metadata\":{}},{\"name\":\"emp_name\",\"type\":\"string\",\"nullable\":false,\"metadata\":{}},{\"name\":\"sal\",\"type\":\"integer\",\"nullable\":false,\"metadata\":{}},{\"name\":\"dept\",\"type\":\"string\",\"nullable\":false,\"metadata\":{}}]}","partitionColumns":[],"configuration":{"description":"table created for demo purpose"},"createdTime":1717525317138}}

In [0]:
%sql 

select * from employee_demo

emp_id,emp_name,sal,dept


#observe after inserting

In [0]:
%sql 

insert into  employee_demo values(1,"varun",13,"sales")

num_affected_rows,num_inserted_rows
1,1


In [0]:

#we can clearly see that 1 more json log file gets stored which holds the latest curd operation
%fs

ls /FileStore/tables/delta/create_table/_delta_log/


path,name,size,modificationTime
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-0,.s3-optimization-0,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-1,.s3-optimization-1,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-2,.s3-optimization-2,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000000.crc,00000000000000000000.crc,2235,1717524476000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000000.json,00000000000000000000.json,1255,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000001.crc,00000000000000000001.crc,2144,1717525322000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000001.json,00000000000000000001.json,1149,1717525318000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000002.crc,00000000000000000002.crc,2094,1717527099000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000002.json,00000000000000000002.json,1049,1717527096000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000003.crc,00000000000000000003.crc,2108,1717527627000


In [0]:
%fs

ls /FileStore/tables/delta/create_table/

path,name,size,modificationTime
dbfs:/FileStore/tables/delta/create_table/_delta_log/,_delta_log/,0,0
dbfs:/FileStore/tables/delta/create_table/part-00000-0652a969-0925-4406-a53c-73a79bf10105-c000.snappy.parquet,part-00000-0652a969-0925-4406-a53c-73a79bf10105-c000.snappy.parquet,1301,1717533344000


In [0]:
%fs

head /FileStore/tables/delta/create_table/part-00000-0652a969-0925-4406-a53c-73a79bf10105-c000.snappy.parquet

PAR1Ň��
$"����8varun����
$"��ܰ	8salesvarunvarunsalessalesFNP�F�P\Hspark_schema%emp_id%emp_name%L%sal%dept%LL&5emp_idBF&<(��.&N5emp_nameLP&N<varunvarun(varunvarun��2&�5salBF&�<(��.&�5deptLP&�<salessales(salessales��2�&�\org.apache.spark.timeZoneEtc/UTCorg.apache.spark.legacyINT96org.apache.spark.version3.3.2)org.apache.spark.sql.parquet.row.metadata�{"type":"struct","fields":[{"name":"emp_id","type":"integer","nullable":true,"metadata":{}},{"name":"emp_name","type":"string","nullable":true,"metadata":{}},{"name":"sal","type":"integer","nullable":true,"metadata":{}},{"name":"dept","type":"string","nullable":true,"metadata":{}}]}org.apache.spark.legacyDateTimeZparquet-mr version 1.12.3-databricks-0009 (build 13debca92ae2826f5c048fb3613d610c77c3bb90)L�PAR1

In [0]:
#we can see that operation:write and mode is append

%fs

head /FileStore/tables/delta/create_table/_delta_log/00000000000000000004.json

{"commitInfo":{"timestamp":1717533343287,"userId":"8432478468657958","userName":"paruchuri7781@gmail.com","operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"notebook":{"notebookId":"1388222939003274"},"clusterId":"0604-175509-m5964sr3","readVersion":3,"isolationLevel":"WriteSerializable","isBlindAppend":true,"operationMetrics":{"numFiles":"1","numOutputRows":"1","numOutputBytes":"1301"},"engineInfo":"Databricks-Runtime/12.2.x-scala2.12","txnId":"5824c69e-e418-4545-a507-38cb3556e7ce"}}
{"add":{"path":"part-00000-0652a969-0925-4406-a53c-73a79bf10105-c000.snappy.parquet","partitionValues":{},"size":1301,"modificationTime":1717533344000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"emp_id\":1,\"emp_name\":\"varun\",\"sal\":13,\"dept\":\"sales\"},\"maxValues\":{\"emp_id\":1,\"emp_name\":\"varun\",\"sal\":13,\"dept\":\"sales\"},\"nullCount\":{\"emp_id\":0,\"emp_name\":0,\"sal\":0,\"dept\":0}}","tags":{"INSERTION_TIME":"1717533344000000","MIN_INSERTION_TIME":"1717533344000000","MAX_INSERTION_TIME":"1717533344000000","OPTIMIZE_TARGET_SIZE":"268435456"}}}

In [0]:
%sql

insert into employee_demo values
(2,"sahithi",14,"hr"),
(3,"kusuma",12,"front_end")

num_affected_rows,num_inserted_rows
2,2


In [0]:
%fs
ls /FileStore/tables/delta/create_table/

path,name,size,modificationTime
dbfs:/FileStore/tables/delta/create_table/_delta_log/,_delta_log/,0,0
dbfs:/FileStore/tables/delta/create_table/part-00000-0652a969-0925-4406-a53c-73a79bf10105-c000.snappy.parquet,part-00000-0652a969-0925-4406-a53c-73a79bf10105-c000.snappy.parquet,1301,1717533344000
dbfs:/FileStore/tables/delta/create_table/part-00000-3fa73c01-89cb-4a7b-a9c5-f0f5382e9a3e-c000.snappy.parquet,part-00000-3fa73c01-89cb-4a7b-a9c5-f0f5382e9a3e-c000.snappy.parquet,1310,1717534516000


In [0]:
#we can clearly see 5th json file got added
%fs

ls dbfs:/FileStore/tables/delta/create_table/_delta_log/

path,name,size,modificationTime
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-0,.s3-optimization-0,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-1,.s3-optimization-1,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-2,.s3-optimization-2,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000000.crc,00000000000000000000.crc,2235,1717524476000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000000.json,00000000000000000000.json,1255,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000001.crc,00000000000000000001.crc,2144,1717525322000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000001.json,00000000000000000001.json,1149,1717525318000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000002.crc,00000000000000000002.crc,2094,1717527099000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000002.json,00000000000000000002.json,1049,1717527096000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000003.crc,00000000000000000003.crc,2108,1717527627000


In [0]:
%fs

head dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000005.json

{"commitInfo":{"timestamp":1717534515946,"userId":"8432478468657958","userName":"paruchuri7781@gmail.com","operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"notebook":{"notebookId":"1388222939003274"},"clusterId":"0604-175509-m5964sr3","readVersion":4,"isolationLevel":"WriteSerializable","isBlindAppend":true,"operationMetrics":{"numFiles":"1","numOutputRows":"2","numOutputBytes":"1310"},"engineInfo":"Databricks-Runtime/12.2.x-scala2.12","txnId":"3603eeed-f430-4b2b-b8f5-357ce48469bb"}}
{"add":{"path":"part-00000-3fa73c01-89cb-4a7b-a9c5-f0f5382e9a3e-c000.snappy.parquet","partitionValues":{},"size":1310,"modificationTime":1717534516000,"dataChange":true,"stats":"{\"numRecords\":2,\"minValues\":{\"emp_id\":2,\"emp_name\":\"kusuma\",\"sal\":12,\"dept\":\"front_end\"},\"maxValues\":{\"emp_id\":3,\"emp_name\":\"sahithi\",\"sal\":14,\"dept\":\"hr\"},\"nullCount\":{\"emp_id\":0,\"emp_name\":0,\"sal\":0,\"dept\":0}}","tags":{"INSERTION_TIME":"1717534516000000","MIN_INSERTION_TIME":"1717534516000000","MAX_INSERTION_TIME":"1717534516000000","OPTIMIZE_TARGET_SIZE":"268435456"}}}

#observe after deletion

In [0]:
%sql
select * from employee_demo

emp_id,emp_name,sal,dept
2,sahithi,14,hr
3,kusuma,12,front_end
1,varun,13,sales


In [0]:
%sql
delete from employee_demo
where emp_id=1

num_affected_rows
1


In [0]:
%fs
ls /FileStore/tables/delta/create_table/

path,name,size,modificationTime
dbfs:/FileStore/tables/delta/create_table/_delta_log/,_delta_log/,0,0
dbfs:/FileStore/tables/delta/create_table/part-00000-0652a969-0925-4406-a53c-73a79bf10105-c000.snappy.parquet,part-00000-0652a969-0925-4406-a53c-73a79bf10105-c000.snappy.parquet,1301,1717533344000
dbfs:/FileStore/tables/delta/create_table/part-00000-3fa73c01-89cb-4a7b-a9c5-f0f5382e9a3e-c000.snappy.parquet,part-00000-3fa73c01-89cb-4a7b-a9c5-f0f5382e9a3e-c000.snappy.parquet,1310,1717534516000


In [0]:
%fs
head /FileStore/tables/delta/create_table/part-00000-3fa73c01-89cb-4a7b-a9c5-f0f5382e9a3e-c000.snappy.parquet

PAR1 ����46:����hsahithikusuma ����424��½8hr	front_endkusumasahithi	front_endhrNVh�N�b\Hspark_schema%emp_id%emp_name%L%sal%dept%LL&5emp_idJN&<(��.&V5emp_namedh&V<6(sahithikusuma��8&�5salJN&�<(��.&�5dept`b&�<6(hr	front_end��4�&�\org.apache.spark.timeZoneEtc/UTCorg.apache.spark.legacyINT96org.apache.spark.version3.3.2)org.apache.spark.sql.parquet.row.metadata�{"type":"struct","fields":[{"name":"emp_id","type":"integer","nullable":true,"metadata":{}},{"name":"emp_name","type":"string","nullable":true,"metadata":{}},{"name":"sal","type":"integer","nullable":true,"metadata":{}},{"name":"dept","type":"string","nullable":true,"metadata":{}}]}org.apache.spark.legacyDateTimeZparquet-mr version 1.12.3-databricks-0009 (build 13debca92ae2826f5c048fb3613d610c77c3bb90)L�PAR1

In [0]:
%fs
ls /FileStore/tables/delta/create_table/_delta_log

path,name,size,modificationTime
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-0,.s3-optimization-0,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-1,.s3-optimization-1,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-2,.s3-optimization-2,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000000.crc,00000000000000000000.crc,2235,1717524476000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000000.json,00000000000000000000.json,1255,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000001.crc,00000000000000000001.crc,2144,1717525322000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000001.json,00000000000000000001.json,1149,1717525318000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000002.crc,00000000000000000002.crc,2094,1717527099000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000002.json,00000000000000000002.json,1049,1717527096000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000003.crc,00000000000000000003.crc,2108,1717527627000


In [0]:
#operation: delete

%fs
head /FileStore/tables/delta/create_table/_delta_log/00000000000000000006.json

{"commitInfo":{"timestamp":1717534977316,"userId":"8432478468657958","userName":"paruchuri7781@gmail.com","operation":"DELETE","operationParameters":{"predicate":"[\"(emp_id#6745 = 1)\"]"},"notebook":{"notebookId":"1388222939003274"},"clusterId":"0604-175509-m5964sr3","readVersion":5,"isolationLevel":"WriteSerializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"1","numRemovedBytes":"1301","numCopiedRows":"0","numDeletionVectorsAdded":"0","numDeletionVectorsRemoved":"0","numAddedChangeFiles":"0","executionTimeMs":"2720","numDeletedRows":"1","scanTimeMs":"1821","numAddedFiles":"0","numAddedBytes":"0","rewriteTimeMs":"886"},"engineInfo":"Databricks-Runtime/12.2.x-scala2.12","txnId":"90e7fc2c-f3ee-4dfd-b31c-e1a8a063f430"}}
{"remove":{"path":"part-00000-0652a969-0925-4406-a53c-73a79bf10105-c000.snappy.parquet","deletionTimestamp":1717534977306,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":1301,"tags":{"INSERTION_TIME":"1717533344000000","MIN_INSERTION_TIME":"1717533344000000","MAX_INSERTION_TIME":"1717533344000000","OPTIMIZE_TARGET_SIZE":"268435456"}}}

#inserting multiple data

In [0]:
%sql
INSERT INTO employee_demo VALUES (4, 'John', 30, 'manager');
INSERT INTO employee_demo VALUES (5, 'Jane', 25, 'developer');
INSERT INTO employee_demo VALUES (6, 'Doe', 35, 'analyst');
INSERT INTO employee_demo VALUES (7, 'Smith', 28, 'sales');
INSERT INTO employee_demo VALUES (8, 'Emily', 32, 'marketing');


num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql

delete from employee_demo where emp_id=6;

num_affected_rows
1


In [0]:
%fs
ls /FileStore/tables/delta/create_table/

path,name,size,modificationTime
dbfs:/FileStore/tables/delta/create_table/_delta_log/,_delta_log/,0,0
dbfs:/FileStore/tables/delta/create_table/part-00000-0652a969-0925-4406-a53c-73a79bf10105-c000.snappy.parquet,part-00000-0652a969-0925-4406-a53c-73a79bf10105-c000.snappy.parquet,1301,1717533344000
dbfs:/FileStore/tables/delta/create_table/part-00000-23bb2086-37b7-47fb-8403-7f075df5d984-c000.snappy.parquet,part-00000-23bb2086-37b7-47fb-8403-7f075df5d984-c000.snappy.parquet,1308,1717536250000
dbfs:/FileStore/tables/delta/create_table/part-00000-3fa73c01-89cb-4a7b-a9c5-f0f5382e9a3e-c000.snappy.parquet,part-00000-3fa73c01-89cb-4a7b-a9c5-f0f5382e9a3e-c000.snappy.parquet,1310,1717534516000
dbfs:/FileStore/tables/delta/create_table/part-00000-54e5463a-104d-47a7-921c-60a256bc7502-c000.snappy.parquet,part-00000-54e5463a-104d-47a7-921c-60a256bc7502-c000.snappy.parquet,1322,1717536254000
dbfs:/FileStore/tables/delta/create_table/part-00000-8eb329b6-3263-4959-ab26-ed7b73287aa2-c000.snappy.parquet,part-00000-8eb329b6-3263-4959-ab26-ed7b73287aa2-c000.snappy.parquet,1329,1717536269000
dbfs:/FileStore/tables/delta/create_table/part-00000-90160768-fbc9-45d5-b561-a77a94bc322e-c000.snappy.parquet,part-00000-90160768-fbc9-45d5-b561-a77a94bc322e-c000.snappy.parquet,1301,1717536258000
dbfs:/FileStore/tables/delta/create_table/part-00000-be827dad-656a-41db-983c-4b0464654ba5-c000.snappy.parquet,part-00000-be827dad-656a-41db-983c-4b0464654ba5-c000.snappy.parquet,1300,1717536261000


In [0]:
%fs
ls /FileStore/tables/delta/create_table/_delta_log/

path,name,size,modificationTime
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-0,.s3-optimization-0,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-1,.s3-optimization-1,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-2,.s3-optimization-2,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000000.crc,00000000000000000000.crc,2235,1717524476000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000000.json,00000000000000000000.json,1255,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000001.crc,00000000000000000001.crc,2144,1717525322000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000001.json,00000000000000000001.json,1149,1717525318000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000002.crc,00000000000000000002.crc,2094,1717527099000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000002.json,00000000000000000002.json,1049,1717527096000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000003.crc,00000000000000000003.crc,2108,1717527627000


In [0]:
%fs
head /FileStore/tables/delta/create_table/_delta_log/00000000000000000012.json

{"commitInfo":{"timestamp":1717536297656,"userId":"8432478468657958","userName":"paruchuri7781@gmail.com","operation":"DELETE","operationParameters":{"predicate":"[\"(emp_id#10305 = 6)\"]"},"notebook":{"notebookId":"1388222939003274"},"clusterId":"0604-175509-m5964sr3","readVersion":11,"isolationLevel":"WriteSerializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"1","numRemovedBytes":"1301","numCopiedRows":"0","numDeletionVectorsAdded":"0","numDeletionVectorsRemoved":"0","numAddedChangeFiles":"0","executionTimeMs":"2570","numDeletedRows":"1","scanTimeMs":"1840","numAddedFiles":"0","numAddedBytes":"0","rewriteTimeMs":"730"},"engineInfo":"Databricks-Runtime/12.2.x-scala2.12","txnId":"37b71b6a-352d-48e8-af7b-03b2f0577ae3"}}
{"remove":{"path":"part-00000-90160768-fbc9-45d5-b561-a77a94bc322e-c000.snappy.parquet","deletionTimestamp":1717536297654,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":1301,"tags":{"INSERTION_TIME":"1717536258000000","MIN_INSERTION_TIME":"1717536258000000","MAX_INSERTION_TIME":"1717536258000000","OPTIMIZE_TARGET_SIZE":"268435456"}}}

In [0]:
display(spark.read.format("parquet").load('/FileStore/tables/delta/create_table/_delta_log/00000000000000000010.checkpoint.parquet'))

txn,add,remove,metaData,protocol,rowIdHighWaterMark
null,null,null,null,"List(1, 2, null, null)",null
null,null,null,"List(1c996daf-f6ac-4644-a420-4bc672c31c2c, null, null, List(parquet, Map()), {""type"":""struct"",""fields"":[{""name"":""emp_id"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""emp_name"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""sal"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""dept"",""type"":""string"",""nullable"":true,""metadata"":{}}]}, List(), Map(), 1717527620630)",null,null
null,null,"List(part-00000-0652a969-0925-4406-a53c-73a79bf10105-c000.snappy.parquet, 1717534977306, false, true, Map(), 1301, null, null)",null,null,null
null,"List(part-00000-23bb2086-37b7-47fb-8403-7f075df5d984-c000.snappy.parquet, Map(), 1308, 1717536250000, false, Map(INSERTION_TIME -> 1717536250000000, MIN_INSERTION_TIME -> 1717536250000000, MAX_INSERTION_TIME -> 1717536250000000, OPTIMIZE_TARGET_SIZE -> 268435456), null, null, {""numRecords"":1,""minValues"":{""emp_id"":4,""emp_name"":""John"",""sal"":30,""dept"":""manager""},""maxValues"":{""emp_id"":4,""emp_name"":""John"",""sal"":30,""dept"":""manager""},""nullCount"":{""emp_id"":0,""emp_name"":0,""sal"":0,""dept"":0}}, List(1, List(4, John, 30, manager), List(4, John, 30, manager), List(0, 0, 0, 0)))",null,null,null,null
null,"List(part-00000-3fa73c01-89cb-4a7b-a9c5-f0f5382e9a3e-c000.snappy.parquet, Map(), 1310, 1717534516000, false, Map(INSERTION_TIME -> 1717534516000000, MIN_INSERTION_TIME -> 1717534516000000, MAX_INSERTION_TIME -> 1717534516000000, OPTIMIZE_TARGET_SIZE -> 268435456), null, null, {""numRecords"":2,""minValues"":{""emp_id"":2,""emp_name"":""kusuma"",""sal"":12,""dept"":""front_end""},""maxValues"":{""emp_id"":3,""emp_name"":""sahithi"",""sal"":14,""dept"":""hr""},""nullCount"":{""emp_id"":0,""emp_name"":0,""sal"":0,""dept"":0}}, List(2, List(2, kusuma, 12, front_end), List(3, sahithi, 14, hr), List(0, 0, 0, 0)))",null,null,null,null
null,"List(part-00000-54e5463a-104d-47a7-921c-60a256bc7502-c000.snappy.parquet, Map(), 1322, 1717536254000, false, Map(INSERTION_TIME -> 1717536254000000, MIN_INSERTION_TIME -> 1717536254000000, MAX_INSERTION_TIME -> 1717536254000000, OPTIMIZE_TARGET_SIZE -> 268435456), null, null, {""numRecords"":1,""minValues"":{""emp_id"":5,""emp_name"":""Jane"",""sal"":25,""dept"":""developer""},""maxValues"":{""emp_id"":5,""emp_name"":""Jane"",""sal"":25,""dept"":""developer""},""nullCount"":{""emp_id"":0,""emp_name"":0,""sal"":0,""dept"":0}}, List(1, List(5, Jane, 25, developer), List(5, Jane, 25, developer), List(0, 0, 0, 0)))",null,null,null,null
null,"List(part-00000-90160768-fbc9-45d5-b561-a77a94bc322e-c000.snappy.parquet, Map(), 1301, 1717536258000, false, Map(INSERTION_TIME -> 1717536258000000, MIN_INSERTION_TIME -> 1717536258000000, MAX_INSERTION_TIME -> 1717536258000000, OPTIMIZE_TARGET_SIZE -> 268435456), null, null, {""numRecords"":1,""minValues"":{""emp_id"":6,""emp_name"":""Doe"",""sal"":35,""dept"":""analyst""},""maxValues"":{""emp_id"":6,""emp_name"":""Doe"",""sal"":35,""dept"":""analyst""},""nullCount"":{""emp_id"":0,""emp_name"":0,""sal"":0,""dept"":0}}, List(1, List(6, Doe, 35, analyst), List(6, Doe, 35, analyst), List(0, 0, 0, 0)))",null,null,null,null
null,"List(part-00000-be827dad-656a-41db-983c-4b0464654ba5-c000.snappy.parquet, Map(), 1300, 1717536261000, false, Map(INSERTION_TIME -> 1717536261000000, MIN_INSERTION_TIME -> 1717536261000000, MAX_INSERTION_TIME -> 1717536261000000, OPTIMIZE_TARGET_SIZE -> 268435456), null, null, {""numRecords"":1,""minValues"":{""emp_id"":7,""emp_name"":""Smith"",""sal"":28,""dept"":""sales""},""maxValues"":{""emp_id"":7,""emp_name"":""Smith"",""sal"":28,""dept"":""sales""},""nullCount"":{""emp_id"":0,""emp_name"":0,""sal"":0,""dept"":0}}, List(1, List(7, Smith, 28, sales), List(7, Smith, 28, sales), List(0, 0, 0, 0)))",null,null,null,null


In [0]:
%sql
update employee_demo
set emp_name="var_sahi"
where emp_id=4

num_affected_rows
1


In [0]:
%sql 

select * from employee_demo

emp_id,emp_name,sal,dept
4,var_sahi,30,manager
8,Emily,32,marketing
5,Jane,25,developer
2,sahithi,14,hr
3,kusuma,12,front_end
7,Smith,28,sales


In [0]:
%fs
ls /FileStore/tables/delta/create_table/

path,name,size,modificationTime
dbfs:/FileStore/tables/delta/create_table/_delta_log/,_delta_log/,0,0
dbfs:/FileStore/tables/delta/create_table/part-00000-0652a969-0925-4406-a53c-73a79bf10105-c000.snappy.parquet,part-00000-0652a969-0925-4406-a53c-73a79bf10105-c000.snappy.parquet,1301,1717533344000
dbfs:/FileStore/tables/delta/create_table/part-00000-23bb2086-37b7-47fb-8403-7f075df5d984-c000.snappy.parquet,part-00000-23bb2086-37b7-47fb-8403-7f075df5d984-c000.snappy.parquet,1308,1717536250000
dbfs:/FileStore/tables/delta/create_table/part-00000-3fa73c01-89cb-4a7b-a9c5-f0f5382e9a3e-c000.snappy.parquet,part-00000-3fa73c01-89cb-4a7b-a9c5-f0f5382e9a3e-c000.snappy.parquet,1310,1717534516000
dbfs:/FileStore/tables/delta/create_table/part-00000-54e5463a-104d-47a7-921c-60a256bc7502-c000.snappy.parquet,part-00000-54e5463a-104d-47a7-921c-60a256bc7502-c000.snappy.parquet,1322,1717536254000
dbfs:/FileStore/tables/delta/create_table/part-00000-87ab6de7-d030-4b89-a009-8d2c9c18c8a8-c000.snappy.parquet,part-00000-87ab6de7-d030-4b89-a009-8d2c9c18c8a8-c000.snappy.parquet,1336,1717537039000
dbfs:/FileStore/tables/delta/create_table/part-00000-8eb329b6-3263-4959-ab26-ed7b73287aa2-c000.snappy.parquet,part-00000-8eb329b6-3263-4959-ab26-ed7b73287aa2-c000.snappy.parquet,1329,1717536269000
dbfs:/FileStore/tables/delta/create_table/part-00000-90160768-fbc9-45d5-b561-a77a94bc322e-c000.snappy.parquet,part-00000-90160768-fbc9-45d5-b561-a77a94bc322e-c000.snappy.parquet,1301,1717536258000
dbfs:/FileStore/tables/delta/create_table/part-00000-be827dad-656a-41db-983c-4b0464654ba5-c000.snappy.parquet,part-00000-be827dad-656a-41db-983c-4b0464654ba5-c000.snappy.parquet,1300,1717536261000


In [0]:
%fs
ls /FileStore/tables/delta/create_table/_delta_log/

path,name,size,modificationTime
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-0,.s3-optimization-0,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-1,.s3-optimization-1,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/.s3-optimization-2,.s3-optimization-2,0,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000000.crc,00000000000000000000.crc,2235,1717524476000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000000.json,00000000000000000000.json,1255,1717524454000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000001.crc,00000000000000000001.crc,2144,1717525322000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000001.json,00000000000000000001.json,1149,1717525318000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000002.crc,00000000000000000002.crc,2094,1717527099000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000002.json,00000000000000000002.json,1049,1717527096000
dbfs:/FileStore/tables/delta/create_table/_delta_log/00000000000000000003.crc,00000000000000000003.crc,2108,1717527627000


In [0]:
%fs
head /FileStore/tables/delta/create_table/_delta_log/00000000000000000013.json

{"commitInfo":{"timestamp":1717537038777,"userId":"8432478468657958","userName":"paruchuri7781@gmail.com","operation":"UPDATE","operationParameters":{"predicate":"[\"(emp_id#11210 = 4)\"]"},"notebook":{"notebookId":"1388222939003274"},"clusterId":"0604-175509-m5964sr3","readVersion":12,"isolationLevel":"WriteSerializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"1","numRemovedBytes":"1308","numCopiedRows":"0","numDeletionVectorsAdded":"0","numDeletionVectorsRemoved":"0","numAddedChangeFiles":"0","executionTimeMs":"3083","scanTimeMs":"1979","numAddedFiles":"1","numUpdatedRows":"1","numAddedBytes":"1336","rewriteTimeMs":"1103"},"engineInfo":"Databricks-Runtime/12.2.x-scala2.12","txnId":"5eb1f5f9-318c-4393-9a24-f7ae6f3f8ca7"}}
{"remove":{"path":"part-00000-23bb2086-37b7-47fb-8403-7f075df5d984-c000.snappy.parquet","deletionTimestamp":1717537038771,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":1308,"tags":{"INSERTION_TIME":"1717536250000000","MIN_INSERTION_TIME":"1717536250000000","MAX_INSERTION_TIME":"1717536250000000","OPTIMIZE_TARGET_SIZE":"268435456"}}}
{"add":{"path":"part-00000-87ab6de7-d030-4b89-a009-8d2c9c18c8a8-c000.snappy.parquet","partitionValues":{},"size":1336,"modificationTime":1717537039000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"emp_id\":4,\"emp_name\":\"var_sahi\",\"sal\":30,\"dept\":\"manager\"},\"maxValues\":{\"emp_id\":4,\"emp_name\":\"var_sahi\",\"sal\":30,\"dept\":\"manager\"},\"nullCount\":{\"emp_id\":0,\"emp_name\":0,\"sal\":0,\"dept\":0}}","tags":{"MAX_INSERTION_TIME":"1717536250000000","INSERTION_TIME":"1717536250000000","MIN_INSERTION_TIME":"1717536250000000","OPTIMIZE_TARGET_SIZE":"268435456"}}}